In [1]:
import re
import pandas as pd
import urllib.request
from tqdm import tqdm
import hgtk
import pandas as pd
from tqdm import tqdm

In [17]:
import fasttext

In [27]:
def word_to_jamo(token):
    def to_special_token(jamo):
        if not jamo:
            return '-'
        else:
            return jamo
    decomposed_token = ''
    for char in token:
        try:
            # char(음절)을 초성, 중성, 종성으로 분리
            cho, jung, jong = hgtk.letter.decompose(char)

            # 자모가 빈 문자일 경우 특수문자 -로 대체
            cho = to_special_token(cho)
            jung = to_special_token(jung)
            jong = to_special_token(jong)
            decomposed_token = decomposed_token + cho + jung + jong

        # 만약 char(음절)이 한글이 아닐 경우 자모를 나누지 않고 추가
        except Exception as exception:
            if type(exception).__name__ == 'NotHangulException':
                decomposed_token += char
    # 단어 토큰의 자모 단위 분리 결과를 추가
    return decomposed_token

In [4]:
csv_dir = '../NIKL_MP_CSV/NXMP1902008040_{}.csv'

In [56]:
with open('tokenized_data.txt', 'w') as out:
    for i in range(5):
        df = pd.read_csv(csv_dir.format(i + 1))
        print(csv_dir.format(i + 1))
        form_series = df['form'].values
        ids = df['id'].values
        forms = ''
        for form, idx in tqdm(zip(form_series, ids)):
            if idx == 1:
                forms = word_to_jamo(forms)
                out.write(forms + '\n')
                forms = ''
            forms += form + ' '

../NIKL_MP_CSV/NXMP1902008040_1.csv


1000000it [01:03, 15647.35it/s]


../NIKL_MP_CSV/NXMP1902008040_2.csv


1000000it [01:04, 15562.78it/s]


../NIKL_MP_CSV/NXMP1902008040_3.csv


1000000it [01:04, 15581.48it/s]


../NIKL_MP_CSV/NXMP1902008040_4.csv


1000000it [01:04, 15492.71it/s]


../NIKL_MP_CSV/NXMP1902008040_5.csv


559602it [00:35, 15559.65it/s]


In [47]:
def jamo_to_word(jamo_sequence):
    tokenized_jamo = []
    index = 0

    # 1. 초기 입력
    # jamo_sequence = 'ㄴㅏㅁㄷㅗㅇㅅㅐㅇ'

    while index < len(jamo_sequence):
        # 문자가 한글(정상적인 자모)이 아닐 경우
        if not hgtk.checker.is_hangul(jamo_sequence[index]):
            tokenized_jamo.append(jamo_sequence[index])
            index = index + 1

        # 문자가 정상적인 자모라면 초성, 중성, 종성을 하나의 토큰으로 간주.
        else:
            tokenized_jamo.append(jamo_sequence[index:index + 3])
            index = index + 3

    # 2. 자모 단위 토큰화 완료
    # tokenized_jamo : ['ㄴㅏㅁ', 'ㄷㅗㅇ', 'ㅅㅐㅇ']

    word = ''
    try:
        for jamo in tokenized_jamo:

            # 초성, 중성, 종성의 묶음으로 추정되는 경우
            if len(jamo) == 3:
                if jamo[2] == "-":
                    # 종성이 존재하지 않는 경우
                    word = word + hgtk.letter.compose(jamo[0], jamo[1])
                else:
                    # 종성이 존재하는 경우
                    word = word + hgtk.letter.compose(jamo[0], jamo[1], jamo[2])
            # 한글이 아닌 경우
            else:
                word = word + jamo

    # 복원 중(hgtk.letter.compose) 에러 발생 시 초기 입력 리턴.
    # 복원이 불가능한 경우 예시) 'ㄴ!ㅁㄷㅗㅇㅅㅐㅇ'
    except Exception as exception:
        if type(exception).__name__ == 'NotHangulException':
            return jamo_sequence

    # 3. 단어로 복원 완료
    # word : '남동생'

    return word

In [45]:
def transform(word_sequence):
    return [(jamo_to_word(word), similarity) for (similarity, word) in word_sequence]

In [57]:
model = fasttext.train_unsupervised('tokenized_data.txt', model='cbow')

In [58]:
model.save_model("fasttext_with_NIKL_MP_CSV.bin") # 모델 저장

In [59]:
model = fasttext.load_model("fasttext_with_NIKL_MP_CSV.bin") # 모델 로드

In [63]:
model.get_input_matrix()

array([[ 1.3172704 , -2.6030622 , -2.6381116 , ...,  4.1049676 ,
        -1.9797415 ,  1.6904285 ],
       [ 8.051485  ,  4.290622  ,  9.813015  , ...,  8.874257  ,
        -5.1378565 , -3.8676076 ],
       [ 2.0295978 ,  1.0167238 , -0.35376626, ...,  2.0631907 ,
        -1.3718491 ,  2.4111595 ],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]], dtype=float32)

In [64]:
model[word_to_jamo('제주도')]

array([-1.05719745e+00, -1.22204649e+00, -8.70347559e-01,  1.03027868e+00,
        1.10074043e+00, -7.09483445e-01,  8.26699436e-02,  9.39434826e-01,
       -6.09719872e-01, -3.80814821e-01,  9.14730355e-02,  3.44184041e-02,
        2.26223215e-01, -8.51046622e-01, -4.20999765e-01,  1.19344115e+00,
        4.79467541e-01,  8.11064914e-02, -6.17881119e-01, -4.45593417e-01,
       -9.48177040e-01,  1.10318832e-01,  2.47502010e-02, -3.92870419e-02,
        1.68602001e-02, -6.93905726e-02, -8.83604646e-01,  2.66338438e-01,
       -5.58378696e-01, -7.66422153e-01, -3.21070224e-01, -2.77237445e-01,
       -1.50739098e+00, -8.00510645e-01, -1.03784405e-01, -1.80575177e-01,
       -1.53279707e-01, -2.30275422e-01, -3.40472497e-02, -4.64885861e-01,
        2.24142209e-01,  1.86688170e-01,  2.81072080e-01,  1.03564394e+00,
        2.44351640e-01,  8.61467645e-02,  1.38720810e-01,  5.05607188e-01,
        1.21292867e-01, -1.07393074e+00, -1.00271070e+00, -6.26787543e-01,
       -4.82832193e-01, -

In [80]:
transform(model.get_nearest_neighbors(word_to_jamo('북극'), k=10))

[('북', 0.8500924110412598),
 ('북녘', 0.810660183429718),
 ('북향', 0.8033188581466675),
 ('뒷북', 0.8031451106071472),
 ('북·중', 0.780265748500824),
 ('북벌', 0.7786989212036133),
 ('북풍', 0.7652722597122192),
 ('북핵', 0.7651371955871582),
 ('북단', 0.7599114775657654),
 ('북촌', 0.757582426071167)]